In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 5.1MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 5.0MB/s 


In [0]:
import pandas as pd
import numpy as np


import xgboost as xgb

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK
from eli5.sklearn import PermutationImportance


In [35]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_Car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_Car


In [36]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

#Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]

  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm3')[0].replace(' ','')))

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [42]:
df['param_wersja']

0                II (2012-)
2             I (2009-2016)
3                II (2010-)
4            T7 (2008-2013)
5               VII (2012-)
                ...        
160609        I (2008-2014)
160610                 None
160611        B (2005-2011)
160614    B5 FL (2000-2005)
160615                 None
Name: param_wersja, Length: 106494, dtype: object

In [44]:
feats =	['param_napęd__cat', 'param_rok-produkcji','param_stan__cat',	'param_skrzynia-biegów__cat','param_faktura-vat__cat',	'param_moc',	'param_marka-pojazdu__cat',	'feature_kamera-cofania__cat','param_typ__cat',	'param_pojemność-skokowa',	'seller_name__cat','feature_wspomaganie-kierownicy__cat',	'param_model-pojazdu__cat','param_wersja__cat',	'param_kod-silnika__cat','feature_system-start-stop__cat',	'feature_asystent-pasa-ruchu__cat',	'feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}
run_model(xgb.XGBRegressor(**xgb_params), feats)



[16:52:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:52:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:52:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

In [47]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

xgb_reg_params = {
      'learning_rate':  hp.choice('learning_rate',   np.arange(0.05, 0.31, 0.05)),
      'max_depth':      hp.choice('max_depth',       np.arange(5,16,1,dtype = int) ),
      'subsample':      hp.quniform('subsample', 0.5, 1, 0.05),
      'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
      'objective':      'reg:squarederror',
      'n_estimators':   100,
      'seed': 0,
}

best = fmin(obj_func, xgb_reg_params, algo = tpe.suggest, max_evals=25)
best

Training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.15000000000000002, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.5}
Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.3, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.15000000000000002, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training with params: 
{'colsample_bytree': 0.700000

{'colsample_bytree': 0.8500000000000001,
 'learning_rate': 1,
 'max_depth': 8,
 'subsample': 0.65}

In [48]:
ls

data/                     Day3_simple_model.ipynb  day5.ipynb  README.md
day2_visualization.ipynb  Day4.ipynb               LICENSE


In [0]:
!git config --global user.email "zuzanna.ogiela@gmail.com"
!git config --global user.name "zuz201"

In [0]:
!git add Day4.ipynb
!git commit -m "model improvements"